In [1]:
import langchain
import getpass
import os


In [2]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key:  ········


## 1. Abordagem de Roteamento de Query usando Function Calling

In [56]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.tools import tool
from langchain_core.runnables import RunnablePassthrough
import pydantic
import pandas as pd
import time

In [37]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=1,
    max_tokens=1000,
    max_retries=2,
)

In [38]:
@tool
def get_movie_recommendation(year: int, genre: str) -> str:
    """Recomenda um filme específico com base no ano e no gênero"""
    return f"Titanic, um filme de {genre} de {year}."

@tool
def query_movie_database(name: str) -> str:
    """Consulta informações sobre um filme específico"""
    return f"O filme {name} tem uma ótima avaliação no IMDb."

@tool
def ensinar_cachorro(name: str) -> str:
    """Ensina um cachorro a ser mais rápido e inteligente"""
    return f"Ensinando o cachorro {name} a pular."

@tool
def fazer_cafe(quantidade: str) -> str:
    """Prepara café em uma quantidade específica"""
    return f"Fazendo {quantidade} de café."

In [39]:
tools = [get_movie_recommendation, query_movie_database, ensinar_cachorro, fazer_cafe]

In [40]:
llm_with_tools = llm.bind_tools(tools)

In [41]:
perguntas = [
    "Me recomenda um filme de ação de 2010?",
    "Quais filmes de ficção científica você recomendaria?",
    "Qual o nome do filme que foi lançado em 2005?",
    "Me fale sobre o filme O Senhor dos Anéis",
    "Como faço para ensinar meu cachorro Spark a ser mais rápido?",
    "Me recomenda um filme de terror?",
    "Você sabe fazer bolacha?",
    "Como recomenda fazer pão?",
    "Me faça 600ml de café",
    "Me fale sobre o filme Titanic",
    "Quero todos os cafés",
    "Me ajude a trocar o pneu",
    "Oi"
]


In [42]:
def generate_answers(perguntas):
    respostas = []
    
    # Para cada pergunta, cria uma mensagem e invoca o modelo
    for pergunta in perguntas:
        time.sleep(1)
        message = HumanMessage(content=pergunta)

        ai_msg = llm_with_tools.invoke([message])
        
        # respostas.append((pergunta, ai_msg))
    
        yield (pergunta, ai_msg)

In [43]:
result = []

In [44]:
for pergunta, resposta in generate_answers(perguntas):
    if len(resposta.tool_calls) > 0:
        for tool_call in resposta.tool_calls:
            selected_tool = {
                    "get_movie_recommendation": get_movie_recommendation, 
                    "query_movie_database": query_movie_database,
                    "ensinar_cachorro": ensinar_cachorro,
                    "fazer_cafe": fazer_cafe
            }[tool_call["name"].lower()]
            # print(selected_tool)
            tool_output = selected_tool.invoke(tool_call["args"])
            result.append(
                {
                    "query_input": str(pergunta),
                    "tool_id": str(tool_call["id"]),
                    "args": str(tool_call["args"]),
                    "name": str(tool_call["name"]),
                    "tool_output": str(tool_output)
                }
            )
    else:
         result.append(
                {
                    "query_input": str(pergunta),
                    "tool_id": None,
                    "args": None,
                    "name": None,
                    "tool_output": None
                }
            )

In [45]:
df = pd.DataFrame(result)

In [46]:
df

,query_input,tool_id,args,name,tool_output
0,Me recomenda um filme de ação de 2010?,ee003b0f-b28b-40b1-af6b-67c92ff4b539,"{'year': 2010.0, 'genre': 'ação'}",get_movie_recommendation,"Titanic, um filme de ação de 2010."
1,Quais filmes de ficção científica você recomen...,None,None,None,None
2,Qual o nome do filme que foi lançado em 2005?,None,None,None,None
3,Me fale sobre o filme O Senhor dos Anéis,8de7edc1-d28f-4483-9f71-769da8d097a9,{'name': 'O Senhor dos Anéis'},query_movie_database,O filme O Senhor dos Anéis tem uma ótima avali...
4,Como faço para ensinar meu cachorro Spark a se...,850a958a-608f-49ea-bae4-4a4c8595f28e,{'name': 'Spark'},ensinar_cachorro,Ensinando o cachorro Spark a pular.
5,Me recomenda um filme de terror?,None,None,None,None
6,Você sabe fazer bolacha?,None,None,None,None
7,Como recomenda fazer pão?,None,None,None,None
8,Me faça 600ml de café,dd719734-8610-4dbb-8a4f-de8d2d148b90,{'quantidade': '600ml'},fazer_cafe,Fazendo 600ml de café.
9,Me fale sobre o filme Titanic,bbc0cb01-6717-420c-9d24-4d080f264566,{'name': 'Titanic'},query_movie_database,O filme Titanic tem uma ótima avaliação no IMDb.


## 2. Abordagem de Roteamento de Query usando VectorSearch

In [103]:
import faiss
import numpy as np
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [104]:
embedding_model = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004", region='us-central1')

In [115]:
examples = {
    "get_movie_recommendation": [
        "Me recomenda um filme de ação?",
        "Qual filme de terror você sugere?",
        "Preciso de uma indicação de comédia."
    ],
    "query_movie_database": [
        "Me fala sobre o filme Interestelar.",
        "Qual é a história de O Poderoso Chefão?",
        "Quem dirigiu Matrix?"
    ],
    "fazer_cafe": [
        "Me ajude a fazer cafe",
        "Quero café",
        "Pode moer café pra mim?"
    ],
    "none": [
        "Oi",
        "Olá",
        "Como você está?"
    ]
}

In [116]:
# Converte os exemplos para embeddings
example_texts = []
example_labels = []
for label, texts in examples.items():
    for text in texts:
        example_texts.append(text)
        example_labels.append(label)

example_embeddings = embedding_model.embed_documents(example_texts)
index = faiss.IndexFlatL2(len(example_embeddings[0]))  # Index FAISS
index.add(np.array(example_embeddings))  # Adiciona os vetores ao índice

In [117]:
def get_movie_recommendation():
    return f"Recomendo um ótimo filme do gênero terror!"

def query_movie_database():
    return f"Aqui estão informações sobre um filme."
    
def fazer_cafe():
    return f"Fazendo café."


In [118]:
# Função de roteamento baseada em similaridade
def route_query(user_input):
    user_embedding = embedding_model.embed_query(user_input)
    distances, indices = index.search(np.array([user_embedding]), 1)  # Busca o mais próximo
    best_match = example_labels[indices[0][0]]

    if best_match == "get_movie_recommendation":
        return get_movie_recommendation()  # Aqui poderíamos extrair o gênero real
    elif best_match == "query_movie_database":
        return query_movie_database()  # Aqui poderíamos extrair o nome real
    elif best_match == "fazer_cafe":
        return fazer_cafe()
    else:
        return "Nenhuma ação necessária."

In [120]:
perguntas = [
    "Me recomenda um filme de ação de 2010?",
    "Quais filmes de ficção científica você recomendaria?",
    "Qual o nome do filme que foi lançado em 2005?",
    "Me fale sobre o filme O Senhor dos Anéis",
    "Como faço para ensinar meu cachorro Spark a ser mais rápido?",
    "Me recomenda um filme de terror?",
    "Você sabe fazer bolacha?",
    "Como recomenda fazer pão?",
    "Me faça 600ml de café",
    "Me fale sobre o filme Titanic",
    "Quero todos os cafés",
    "Me ajude a trocar o pneu",
    "Oi"
]


In [122]:
for pergunta in perguntas:
    print(route_query(pergunta))

Recomendo um ótimo filme do gênero terror!
Recomendo um ótimo filme do gênero terror!
Aqui estão informações sobre um filme.
Aqui estão informações sobre um filme.
Nenhuma ação necessária.
Recomendo um ótimo filme do gênero terror!
Fazendo café.
Fazendo café.
Fazendo café.
Aqui estão informações sobre um filme.
Fazendo café.
Fazendo café.
Nenhuma ação necessária.
